In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib
# matplotlib.use("TkAgg")  # Or "Qt5Agg" if TkAgg is not available

import matplotlib.pyplot as plt

import pickle
import os

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable GPU

data = pd.read_csv('translation_train_dataset.csv')

# Display the first few rows
print(data.head())

# Parameters
max_length = 40  # Maximum length of sequences
num_samples = len(data)  # Total number of samples

# Initialize Tokenizer
english_tokenizer = Tokenizer()
tulu_tokenizer = Tokenizer()

# Fit on English and Tulu sentences
english_tokenizer.fit_on_texts(data['English'])
tulu_tokenizer.fit_on_texts(data['Tulu'])

# Save the tokenizers
with open('english_tokenizer.pkl', 'wb') as file:
    pickle.dump(english_tokenizer, file)

with open('tulu_tokenizer.pkl', 'wb') as file:
    pickle.dump(tulu_tokenizer, file)

# Convert sentences to sequences
english_sequences = english_tokenizer.texts_to_sequences(data['English'])
tulu_sequences = tulu_tokenizer.texts_to_sequences(data['Tulu'])

# Pad sequences
english_sequences = pad_sequences(english_sequences, maxlen=max_length, padding='post')
tulu_sequences = pad_sequences(tulu_sequences, maxlen=max_length, padding='post')

# Prepare the input and output
X = english_sequences
y = tulu_sequences

# Prepare the output by shifting
y_input = y[:, :-1]  # Input sequences for the model
y_output = y[:, 1:]  # Output sequences (shifted)

# Before model fitting, print the shapes to debug
print(f'X shape: {X.shape}')  # (num_samples, max_length)
print(f'y_output shape: {y_output.shape}')  # (num_samples, max_length, num_classes)

# Ensure the output sequences are padded to match the model's output shape
y_output = pad_sequences(y_output, maxlen=max_length, padding='post')

# One-hot encoding the output
y_output = tf.keras.utils.to_categorical(y_output, num_classes=len(tulu_tokenizer.word_index) + 1)

# Verify shapes
print(f'X shape: {X.shape}')          # Should be (num_samples, max_length)
print(f'y_output shape: {y_output.shape}')  # Should be (num_samples, max_length, num_classes)

def create_transformer_model(input_dim, output_dim, embedding_dim=1024, num_heads=16, ff_dim=1024, dropout_rate = 0.2):
    # Input layer
    inputs = tf.keras.Input(shape=(None,))

    # Embedding layer
    x = tf.keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_dim)(inputs)

    # Transformer layers
    for _ in range(8):
        attn_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(x, x)
        attn_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn_output + x)

        # Adjust Dense layer to match `embedding_dim`
        x = tf.keras.layers.Dense(ff_dim, activation='relu')(attn_output)
        x = tf.keras.layers.Dropout(dropout_rate)(x)
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

    # Output layer
    outputs = tf.keras.layers.Dense(output_dim, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

# Create the model
model = create_transformer_model(len(english_tokenizer.word_index) + 1, len(tulu_tokenizer.word_index) + 1)

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

initial_learning_rate = 1e-4
decay_steps = 10000
warmup_steps = 4000
# Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    [warmup_steps], [initial_learning_rate / warmup_steps, initial_learning_rate])

# Compile the model with the custom learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_translation_model.keras', save_best_only=True)


# Train the model
history = model.fit(X, y_output,
                    epochs=100,  # Increased epochs for better training
                    batch_size=64,  # Common batch size
                    validation_split=0.2,  # Use 20% of the data for validation
                    callbacks=[early_stopping, model_checkpoint])


# Plot training & validation loss values
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Save the final model
model.save('translation_model.keras')


                                             English  \
0                   The change was written by Chandu   
1              God Gives Everything When He Gives It   
2       Also, the pain of the mind does not go away.   
3                       The potential is on the move   
4  There was a science that we did not understand...   

                                             Tulu  
0                ಬದಲಾವಣೆ ಬರೆಯಿನಾರ್ ಚಂದು ಪನ್ಪಿನಾರ್  
1                     ದೇವೆರ್ ಕೊರ್ನಗ ಮಾತಲ ಕೊರ್ಪೆರ್  
2            ಅಂಚನೆ ಮನಸುಗು ಆಯಿನ ಬೇನೆ ಆ ಬೇನೆ ಮಾಜುಜಿ  
3                               ವಿಭವೆ ನಡತೊಂದುಲ್ಲೆ  
4  ನಮಕ್ ಸೊಲಬೊಗು ಅರ್ಥ ಆವಂದಿನ ಒಂಜಿ ವಿಜ್ಞಾನ ಇತ್ತ್ಂಡ್  
X shape: (8300, 40)
y_output shape: (8300, 39)
X shape: (8300, 40)
y_output shape: (8300, 40, 16949)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None